In [1]:
import os
os.environ["VLLM_ATTENTION_BACKEND"] = "FLASH_ATTN"
from IPython.display import Markdown, display
from tqdm.notebook import tqdm

from llama_index.core import SimpleDirectoryReader, load_index_from_storage, StorageContext, VectorStoreIndex
from llama_index.readers.file import (
    DocxReader,
    PptxReader,
    XMLReader,
    PyMuPDFReader,
    SugoojiReader
)
from llama_index.llms.vllm import Vllm
# from llama_index.llms.huggingface import HuggingFaceLLM

In [2]:
dataset_dir = os.getcwd() + "/datasets"
storage_dir = os.getcwd() + "/storage"

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
if not os.path.exists(storage_dir):
    os.makedirs(storage_dir)

In [3]:
parse = True
if parse == True:
    # Parse Data
    parser = SugoojiReader()
    # parser = PyMuPDFReader()
    file_extractor = {".pdf": parser}
    documents = SimpleDirectoryReader(
        dataset_dir, file_extractor=file_extractor
    ).load_data()
    
    print(documents)
    
    # Saving to VectorDB
    index = VectorStoreIndex.from_documents(documents)
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Saved high-res image at /home/mikequ/workspace/SemenRag/seaRAG/datasets/2025Q1/0.png
/home/mikequ/workspace/SemenRag/seaRAG/datasets/2025Q1/0.png
```markdown
# Farm Fresh Berhad

**Registration Number:** 201001010221 (894851-U)

**(Incorporated in Malaysia)**

## Interim Financial Report

**First quarter ended 30 June 2024**
```
Saved high-res image at /home/mikequ/workspace/SemenRag/seaRAG/datasets/2025Q1/1.png
/home/mikequ/workspace/SemenRag/seaRAG/datasets/2025Q1/1.png
```markdown
# Farm Fresh Berhad

**Registration Number:** 201001010221 (894851-U)

**Incorporated in Malaysia**

## Condensed consolidated statement of financial position as at 30 June 2024

| Unaudited | Audited |
| --- | --- |
| **30.6.2024** | **31.3.2024** |
| **RM'000** | **RM'000** |

### Assets

| Property, plant and equipment | 512,664 | 494,633 |
| Right-of-use assets | 32,742 | 33,124 |
| Biological assets | 135,481 | 133,284 |
| Intangible assets | 113,057 | 112,826 |
| Trade and other receivables | 468 | 5

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

unloaded vlm and loaded llm
[{'role': 'system', 'content': "You are a helpful assistant for parsing markdown into a vector database. Output plain text markdown as is and include the source of the information. When you encounter tables, for every single cell of the table's contents, you need to include a comprehensive description on what the value is and what it represents in NATURAL LANGUAGE, along with the source on a new line. Use your discretion."}, {'role': 'user', 'content': '```markdown\n# Farm Fresh Berhad\n\n**Registration Number:** 201001010221 (894851-U)\n\n**(Incorporated in Malaysia)**\n\n## Interim Financial Report\n\n**First quarter ended 30 June 2024**\n```'}]
['# Farm Fresh Berhad\n\n**Registration Number:** 201001010221 (894851-U)\n\n**(Incorporated in Malaysia)**\n\n## Interim Financial Report\n\n**First quarter ended 30 June 2024**\n\nThe title "Interim Financial Report" indicates that this document provides financial information for a specific period before the end 

In [3]:
# Load VectorDB
storage_context = StorageContext.from_defaults(persist_dir="storage")
index = load_index_from_storage(storage_context, index_id="vector_index")

In [4]:
# LLM
llm = Vllm(model='meta-llama/Meta-Llama-3-8B-Instruct', tensor_parallel_size=1)
# llm = Vllm(model='meta-llama/Meta-Llama-3-70B-Instruct', tensor_parallel_size=4)
# llm = Vllm(model='nvidia/Llama-3.1-Nemotron-70B-Instruct-HF', tensor_parallel_size=4)

# llm = Vllm(model='meta-llama/Meta-Llama-3-70B-Instruct', tensor_parallel_size=4)
# llm = Vllm(model='deepseek-ai/DeepSeek-R1-Distill-Llama-70B', tensor_parallel_size=4)
# llm = HuggingFaceLLM(
#     model_name='nvidia/Llama-3.1-Nemotron-70B-Instruct-HF',  # or another Hugging Face model of your choice
#     device_map="auto"    # Automatically map model layers to available devices (GPU/CPU)
# )

INFO 02-24 23:57:31 __init__.py:190] Automatically detected platform cuda.
INFO 02-24 23:57:35 config.py:542] This model supports multiple tasks: {'reward', 'embed', 'score', 'classify', 'generate'}. Defaulting to 'generate'.
INFO 02-24 23:57:35 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, 

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 02-24 23:57:38 model_runner.py:1115] Loading model weights took 14.9595 GB
INFO 02-24 23:57:40 worker.py:267] Memory profiling takes 2.10 seconds
INFO 02-24 23:57:40 worker.py:267] the current vLLM instance can use total_gpu_memory (23.57GiB) x gpu_memory_utilization (0.90) = 21.21GiB
INFO 02-24 23:57:40 worker.py:267] model weights take 14.96GiB; non_torch_memory takes 0.01GiB; PyTorch activation peak memory takes 1.23GiB; the rest of the memory reserved for KV Cache is 5.01GiB.
INFO 02-24 23:57:41 executor_base.py:110] # CUDA blocks: 2565, # CPU blocks: 2048
INFO 02-24 23:57:41 executor_base.py:115] Maximum concurrency for 8192 tokens per request: 5.01x
INFO 02-24 23:57:42 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_uti

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:12<00:00,  2.72it/s]

INFO 02-24 23:57:54 model_runner.py:1562] Graph capturing finished in 13 secs, took 0.22 GiB
INFO 02-24 23:57:54 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 16.23 seconds


In [5]:
chat_engine = index.as_chat_engine(chat_mode="best")
# chat_engine = index.as_chat_engine(chat_mode="best", verbose=True)

In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        break  # Exit the loop

    streaming_response = chat_engine.stream_chat(user_input)
    
    markdown_response = f"**User:** {user_input}\n\n**LLM:** "  # Format for Markdown

    for chunk in streaming_response.response_gen:
        markdown_response += chunk  # Append chunks to build the full response

    display(Markdown(markdown_response))  # Display in Markdown format

**User:** can you list our non-current assets?

**LLM:** The non-current assets include property, plant and equipment, right-of-use assets, biological assets, intangible assets, and deferred tax assets.

**User:** can you list the associated unaudited value of these assets?

**LLM:** The unaudited value of the non-current assets is 803,270.

**User:** on which table did you get this information?

**LLM:** The information about the unaudited value of non-current assets was retrieved from the financial statements table.

**User:** can you list the value of each item in the non-current assets class

**LLM:** The value of each item in the non-current assets class is as follows:
- Property, plant, and equipment: RM 512,664,000
- Right-of-use assets: RM 32,742,000
- Biological assets: RM 135,481,000
- Intangible assets: RM 113,057,000
- Deferred tax assets: Value not provided in the context information.

**User:** can you provide a summary of our operations in malaysia and australia?

**LLM:** Summary of operations in Malaysia:
- Activities include rearing of dairy cows and goats, production and marketing of cow's and goat's milk, yogurt products, plant-based, ice-cream, and other products, as well as agrotourism.
- Segment profit before tax for Malaysia:
  - Period from April 1, 2024, to June 30, 2024: 28,606
  - Period from April 1, 2023, to June 30, 2023: 9,075
- Revenue from external customers:
  - Period from April 1, 2024, to June 30, 2024: 203,511
  - Period from April 1, 2023, to June 30, 2023: 149,847

Summary of operations in Australia:
- Revenue from external customers: 38,188
- Inter-segment revenue: 24,228
- Depreciation and amortization: (2,228)
- Finance costs: (812)
- Finance income: 81

**User:** in which sections did you obtain this information?

**LLM:** The information about the summary of operations in Malaysia and Australia was obtained from the segment information section of the financial statements.

**User:** what is the section name?

**LLM:** The section name where the information about the summary of operations in Malaysia and Australia was obtained is the "Segment Information" section of the financial statements.